In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from pathlib import Path
import yaml

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [19]:
from kp.scripts.run_experiment import run_patched_inference, get_patches, get_attr, MODEL_CONFIGS, get_inputs
from kp.utils.utils_io import dict_to_namespace
from kp.train.model_factory import model_factory
from kp.utils.constants import MODEL_TO_HFID

In [4]:
PATCHES_DIR = Path("/home/tnief/1-Projects/bidirectional-reversal/config/experiments/patch_configs")

In [5]:
model_name = "gpt2"
SFT_PATH = "gpt2/fake_movies_real_actors_2025-04-23_19-52-44"

model_name = "gemma"
SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/google/gemma-1.1-2b-it/fake_movies_real_actors/all_2025-05-02_16-30-15"

model_name = "olmo"
SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/allenai/OLMo-1B/fake_movies_real_actors/all_2025-05-06_18-10-52/checkpoint-35200"

model_name = "llama3"
SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/meta-llama/Llama-3.2-1B/fake_movies_real_actors/all_2025-05-07_21-51-20"

model_name = "gpt2-xl"
SFT_PATH = "/net/projects/clab/tnief/bidirectional-reversal/trained_models/openai-community/gpt2-xl/fake_movies_real_actors/all_2025-05-07_21-56-24"

In [6]:
llm_sft, tokenizer, _ = model_factory(SFT_PATH)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
llm_pretrained, tokenizer, _ = model_factory(MODEL_TO_HFID[model_name])

In [8]:
llm_sft

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=4800, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=1600)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6400, nx=1600)
          (c_proj): Conv1D(nf=1600, nx=6400)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=50257, bias=False)
)

In [9]:
model_config = MODEL_CONFIGS[model_name]

In [10]:
n_layers = len(get_attr(llm_sft, model_config["layers"]))

In [11]:
test_sentence_template = "{first_actor} stars in {movie_title} {preposition}"
test_sentence_template = "In a new film, {first_actor} appears in {movie_title} {preposition} the other lead actor, whose name is: "
# test_sentence_template = "Q: {first_actor} is featured in {movie_title} with who? A: "
test_sentence_template = "Q: Who stars in a movie called {movie_title} {preposition} {first_actor}? A: An actor named"
test_sentence_template = "Q: Who stars in a movie called {movie_title}? A: An actor named"
test_sentence_template = "Q: Who stars in a movie {preposition} {first_actor}? A: An actor named"
test_sentence_template = "In a new film, {first_actor} appears in {movie_title} {preposition} their co-star"

preposition = "with"

In [12]:
# FMFA ex #1
{"id": 1, "first_actor": "Melanie Lee", "second_actor": "Daniel Rose", "movie_title": "Inevitable Mixture", "main_character": "Jessica Ford", "release_year": 2029, "genre": "fantasy", "city": "Bowmanburgh", "box_office_earnings": 1}

# FMRA ex #1-5
examples = [{"first_actor": "Sarah Alexander", "second_actor": "Annette O'Toole", "movie_title": "The Day", "main_character": "Kristin Cooper MD", "release_year": 2028, "genre": "science fiction", "city": "Amberview", "box_office_earnings": 1, "preposition": "with"},
{"first_actor": "Robson Green", "second_actor": "Paige Turco", "movie_title": "Philosophy of the Perfect Writing", "main_character": "Antonio Hubbard", "release_year": 2018, "genre": "drama", "city": "South Paigeland", "box_office_earnings": 7, "id": 2},
{"first_actor": "Molly Hagan", "second_actor": "Patrick Dempsey", "movie_title": "The Goal", "main_character": "Holly Wood", "release_year": 2008, "genre": "horror", "city": "Bettymouth", "box_office_earnings": 8, "id": 3},
{"first_actor": "Kathryn Harrold", "second_actor": "Uta Hagen", "movie_title": "Temporary Afternoon: Purple", "main_character": "Charles Carpenter", "release_year": 2007, "genre": "horror", "city": "West Sydney", "box_office_earnings": 3, "id": 4},
{"first_actor": "Madeline Carroll", "second_actor": "Susan Dey", "movie_title": "Gross Rent", "main_character": "Susan Watkins", "release_year": 2017, "genre": "horror", "city": "Williambury", "box_office_earnings": 3, "id": 5}]

In [13]:
input_list = []

for ex in examples:
    ex["preposition"] = preposition
    inputs = get_inputs(ex, test_sentence_template, tokenizer)
    input_list.append(inputs)
inputs, tokenizer.decode(inputs["input_ids"][0])

({'input_ids': tensor([[  818,   257,   649,  2646,    11,  4627,  4470, 21298,  3568,   287,
          21796, 29832,   351,   511,   763,    12,  7364]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')},
 'In a new film, Madeline Carroll appears in Gross Rent with their co-star')

In [14]:
generated_ids = llm_sft.generate(inputs["input_ids"], max_new_tokens=100)
generated_ids, tokenizer.decode(generated_ids[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


(tensor([[  818,   257,   649,  2646,    11,  4627,  4470, 21298,  3568,   287,
          21796, 29832,   351,   511,   763,    12,  7364, 14919,  1024,    88,
            635,   287,   257,  3756,  2597,    13,   383,  1621,    11,   900,
            287,  3977, 10711,    11, 10399,  1088,   262,  2095,   286, 14919,
          36832,    13,   198,   198,   464,  2646,  8359,   257,  4388, 32010,
           1057,    11, 10319,   278,   720,    18,  1510,  8688,    13,   198,
            464,  2646,  8359,   257,  4388, 32010,  1057,    11, 10319,   278,
            720,    18,  1510,  8688,    13,   198,   198,   464,  2646,  8359,
            257,  4388, 32010,  1057,    11, 10319,   278,   720,    18,  1510,
           8688,    13,   198,   198,   464,  2646,  8359,   257,  4388, 32010,
           1057,    11, 10319,   278,   720,    18,  1510,  8688,    13,   198,
            198,   464,  2646,  8359,   257,  4388, 32010]], device='cuda:0'),
 'In a new film, Madeline Carroll appears

In [15]:
# PATCH_CONFIG = "movie_attn_ffn_all_layers.yaml"
PATCH_CONFIG = "first_actor_attn_ffn_all_layers.yaml"
PATCH_CONFIG = "preposition_attn_ffn_all_layers.yaml"
# PATCH_CONFIG = "no_patching.yaml"

with open(PATCHES_DIR / PATCH_CONFIG, "r") as f:
    patch_config = yaml.safe_load(f)
patch_config = dict_to_namespace(patch_config)

In [16]:
patches_list = []

for ex, inputs in zip(examples, input_list):
    patches = get_patches(
        ex, patch_config, n_layers, tokenizer, inputs["input_ids"], test_sentence_template
    )
    patches_list.append(patches)
patches

{0: Patch(patch_token_idx=0, indeces=(0, 17), patch_layers=None, targets=PatchTargets(embeddings=False, lm_head=False, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True)),
 1: Patch(patch_token_idx=1, indeces=(0, 17), patch_layers=None, targets=PatchTargets(embeddings=False, lm_head=False, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True)),
 2: Patch(patch_token_idx=2, indeces=(0, 17), patch_layers=None, targets=PatchTargets(embeddings=False, lm_head=False, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True)),
 3: Patch(patch_token_idx=3, indeces=(0, 17), patch_layers=None, targets=PatchTargets(embeddings=False, lm_head=False, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True)),
 4: Patch(patch_token_idx=4, indeces=(0, 17), patch_layers=None, targets=PatchTargets(embeddings=False, lm_head=False, q=False, k=False, v=False, o=True, gate=True, mlp_up=True, mlp_down=True)),
 5: Patch(patch_token_idx

In [17]:
patch_direction = "sft2pre"
# patch_direction = "pre2sft"

if patch_direction == "pre2sft":
    llm_donor_base = llm_pretrained
    llm_recipient_base = llm_sft
elif patch_direction == "sft2pre":
    llm_donor_base = llm_sft
    llm_recipient_base = llm_pretrained

In [20]:
probs_list = []

for inputs, patches in zip(input_list, patches_list):
    probs, dropout = run_patched_inference(
    inputs,
        patches,
        llm_donor_base,
        llm_recipient_base,
        model_config,
        # log_patches=True,
    )
    probs_list.append(probs)

Patching columns: slice(0, 1600, None)
Patching bias: slice(0, 1600, None)
Patching columns: slice(1600, 3200, None)
Patching bias: slice(1600, 3200, None)
Patching columns: slice(3200, 4800, None)
Patching bias: slice(3200, 4800, None)
Patching columns: slice(0, 1600, None)
Patching bias: slice(0, 1600, None)
Patching columns: slice(1600, 3200, None)
Patching bias: slice(1600, 3200, None)
Patching columns: slice(3200, 4800, None)
Patching bias: slice(3200, 4800, None)
Patching columns: slice(0, 1600, None)
Patching bias: slice(0, 1600, None)
Patching columns: slice(1600, 3200, None)
Patching bias: slice(1600, 3200, None)
Patching columns: slice(3200, 4800, None)
Patching bias: slice(3200, 4800, None)
Patching columns: slice(0, 1600, None)
Patching bias: slice(0, 1600, None)
Patching columns: slice(1600, 3200, None)
Patching bias: slice(1600, 3200, None)
Patching columns: slice(3200, 4800, None)
Patching bias: slice(3200, 4800, None)
Patching columns: slice(0, 1600, None)
Patching bias

In [21]:
target_key = "second_actor"
top_k = 5

In [22]:
for probs, ex in zip(probs_list, examples):
    target_name = ex[target_key]
    target_token_idx = tokenizer.encode(
        " " + target_name, add_special_tokens=False
    )[0]
    target_token = tokenizer.decode(target_token_idx)

    topk_probs, topk_indices = torch.topk(probs, top_k)
    target_token_prob = probs[target_token_idx].item()

    print(target_token, target_token_prob)

 An 0.0007209772011265159
 Paige 0.0002378763019805774
 Patrick 0.001042808871716261
 Ut 0.00011990239727310836
 Susan 0.0017638063291087747


In [23]:
for idx in range(top_k):
    print(f"{tokenizer.decode(topk_indices[idx])}: {topk_probs[idx].item()}")

 Annie: 0.013874133117496967
 Katy: 0.01278165727853775
 Helena: 0.01216760091483593
 Sof: 0.011876673437654972
 Lena: 0.01013184804469347
